In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.transforms import v2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pickle
import gc
import os
from tqdm import tqdm
from torchsummary import summary

In [2]:
torch.manual_seed(42)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
data_transform = v2.Compose([
    v2.Resize((180, 180)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize([0.5], [0.5]),
    v2.RandomInvert() 
])

In [5]:

data = datasets.ImageFolder("data/kagglecatsanddogs_5340/PetImages", transform = data_transform)

In [6]:
torch.manual_seed(42)
train_data, test_data = random_split(data, [.7, .3])
train_loader = DataLoader(dataset= train_data, num_workers= 12, batch_size= 512, shuffle= True)
test_loader  = DataLoader(dataset= test_data,  num_workers= 12, batch_size= 512, shuffle= True)

In [7]:
class Cats_n_Dogs_net(nn.Module):
    def __init__(self, random_state):
        super(Cats_n_Dogs_net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels=32, kernel_size = (3,3)) # -4
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2,2))# // 2
        
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels=64, kernel_size=(3,3)) # -2
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2,2))# // 2
        
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = (3,3)) # -2
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=(2,2))# // 2
        
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        self.relu4 = nn.ReLU()
        self.maxpool4 = nn.MaxPool2d(kernel_size=(2,2))
        
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3)
        self.relu5 = nn.ReLU()
        
        torch.manual_seed(random_state)
        self.dropout1 = nn.Dropout2d(p = .5)
        self.flatten1 = nn.Flatten(1)
        
        self.linear1 = nn.LazyLinear(out_features=256)
        self.relu2 = nn.ReLU()
        
        self.dropout2 = nn.Dropout2d(p = 0.2)
        self.linear2 = nn.Linear(in_features = 256, out_features = 2)
        self.softmax1 = nn.Softmax(dim = 1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)
        
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.maxpool4(x)
        
        x = self.conv5(x)
        x = self.relu5(x)
        
        x = self.dropout1(x)
        x = self.flatten1(x)
        
        x = self.linear1(x)
        x = self.relu2(x)
        
        x = self.dropout2(x)
        x = self.linear2(x)
        x = self.softmax1(x)
        return x

In [8]:
model = Cats_n_Dogs_net(42).to(device)
summary(model, (3,180, 180))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 178, 178]             896
              ReLU-2         [-1, 32, 178, 178]               0
         MaxPool2d-3           [-1, 32, 89, 89]               0
            Conv2d-4           [-1, 64, 87, 87]          18,496
              ReLU-5           [-1, 64, 87, 87]               0
         MaxPool2d-6           [-1, 64, 43, 43]               0
            Conv2d-7          [-1, 128, 41, 41]          73,856
              ReLU-8          [-1, 128, 41, 41]               0
         MaxPool2d-9          [-1, 128, 20, 20]               0
           Conv2d-10          [-1, 256, 18, 18]         295,168
             ReLU-11          [-1, 256, 18, 18]               0
        MaxPool2d-12            [-1, 256, 9, 9]               0
           Conv2d-13            [-1, 256, 7, 7]         590,080
             ReLU-14            [-1, 25

/home/pathetic/anaconda3/envs/torch_hub/lib/python3.12/site-packages/torch/nn/functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [9]:
def accuracy(fx, y):
    return (torch.argmax(fx, dim = 1) == y).float().sum()

In [10]:
def fit(model, train_data, test_data, loss_fn = nn.CrossEntropyLoss(), optimizer = optim.Adam, epochs = 500):
    optimizer = optimizer(model.parameters())
    loss_per_epoch = []
    acc_per_epoch = []
    for e in tqdm(range(epochs)):
        running_test_loss = running_train_loss = 0.
        test_acc = train_acc = 0.
        test_size = train_size = 0
        
        model.train()
        for i, (x_train, y_train) in enumerate(train_data):
            x_train = x_train.type(torch.float32).to(device)
            y_train = y_train.to(device)
            
            train_preds = model(x_train)
            train_loss = loss_fn(train_preds, y_train)
            running_train_loss += train_loss.item()
            train_acc += accuracy(train_preds, y_train)
            train_size += x_train.shape[0]
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
        
        model.eval()
        with torch.inference_mode():   
            for j, (x_test, y_test) in enumerate(test_data):
                x_test = x_test.type(torch.float32).to(device)
                y_test = y_test.to(device)
                test_preds = model(x_test)
                running_test_loss += loss_fn(test_preds, y_test)
                test_acc += accuracy(test_preds, y_test)
                test_size += x_test.shape[0]
            print(f"Progress:\n\tTrain loss: {running_train_loss / i} | Train accuracy: {train_acc / train_size}\n\t Test loss: {running_test_loss / j} | Test accuracy: {test_acc / test_size}", end = '\r')
            

In [11]:
import warnings
warnings.simplefilter("ignore")
gc.collect()
torch.cuda.empty_cache()
fit(model, train_loader, test_loader, epochs=50)

  2%|▏         | 1/50 [00:20<17:01, 20.85s/it]

Progress:
	Train loss: 0.7111415687729331 | Train accuracy: 0.5348877310752869


  4%|▍         | 2/50 [00:40<16:19, 20.40s/it]

Progress:
	Train loss: 0.6830433642162996 | Train accuracy: 0.6036345362663269


  6%|▌         | 3/50 [01:01<15:59, 20.41s/it]

...

Progress:
	Train loss: 0.3723902982823989 | Train accuracy: 0.9498257040977478


 98%|█████████▊| 49/50 [16:41<00:19, 19.70s/it]

Progress:
	Train loss: 0.3687946989255793 | Train accuracy: 0.9549117088317871


100%|██████████| 50/50 [17:00<00:00, 20.41s/it]

Progress:
	Train loss: 0.3678039689274395 | Train accuracy: 0.9564546942710876


torch.save(model.state_dict(), "cnd_v2_89_acc.pt")